# Modify metadata - add plot metadata

## Set up

In [ ]:
import os
from irods.session import iRODSSession

cred_path = "D:/iRods_credentials.txt" #file containing iRODS credentials
irods_home = "/tempZone/home/garys" #iRODS home collection
sourceTree = "Y:/garys" #same iRODS home collection, but mapped as a disc drive with davrods

## Get an iRODS session

In [ ]:
creds = [line.rstrip('\n') for line in open(cred_path, "r")]

with iRODSSession(host=creds[0], port=creds[1], user=creds[2], password=creds[3], zone=creds[4], \
                  #client_user=creds[5], \
                  client_server_negotiation = "request_server_negotiation", client_server_policy = "CS_NEG_REQUIRE", \
                  encryption_algorithm = "AES-256-CBC", encryption_key_size = 32, encryption_num_hash_rounds = 16, \
                  encryption_salt_size = 8) as session:
    print ('Got A Session', session)
    pass

collection = session.collections.get(irods_home)
print(collection)

In [ ]:
class Data:
    
    def __init__(self, filename):
        # initialise the experimental data by reading the header from the data (.dat) file
        self.process = True # should this experiment be processed?
        self.axes = []      # axes labels
        self.columns = {}   # column labels and index numbers
        with open(filename) as f:
            lines = [line.rstrip('\n') for line in open(filename)]
            line = 3
            col_no = 1
            while lines[line].find("Column") != -1:
                if lines[line+3].find("type: coordinate") > 0:
                    axis_text = lines[line+1][lines[line+1].find(": ")+2:]
                    self.axes.append(axis_text)
                    line += 4
                elif lines[line+2].find("type: value") > 0:
                    column_text = lines[line+1][lines[line+1].find(": ")+2:]
                    self.columns[column_text]=col_no
                    line += 3
                col_no += 1
        if len(self.axes) < 2: self.process = False

## Loop over Collections

In [ ]:
for sourceDir, subdirList, fileList in os.walk(sourceTree):
    for fname in fileList:
        if fname.endswith(".dat"):
            try:
                d = Data(os.path.join(sourceDir,fname))
                sourceColl = irods_home+sourceDir[len(sourceTree):]
                sourceColl = sourceColl.replace('\\', '/')
                #print(sourceColl)
                collection = session.collections.get(sourceColl)
                #print(sourceDir+" number of axes is "+str(len(d.axes))+" number of columns is "+str(len(d.columns)))
                try:
                    collection.metadata.add("no_of_axes",str(len(d.axes)))
                except:
                    print(sourceDir+" could not add metadata number_of_axes "+str(len(d.axes)))
                try:
                    collection.metadata.add("no_of_data_columns",str(len(d.columns)))
                except:
                    print(sourceDir+" could not add metadata no_of_data_columns "+str(len(d.axes)))
                try:
                    metadata = collection.metadata.get_one("number_of_plots")
                    collection.metadata.remove("number_of_plots",metadata.value)
                except:
                    print(sourceDir+" could not remove metadata number_of_plots")
                for col_text, col_no in d.columns.items():
                    gfile = irods_home+sourceDir[len(sourceTree):]+"/"+fname+"_plot_"+str(col_no)+".jpg"
                    gfile = gfile.replace('\\', '/')
                    try:                        
                        data_object = session.data_objects.get(gfile)
                        data_object.metadata.add("plot_label",col_text)
                    except:
                        print("SOMETHING ODD "+gfile)
            except:
                print(sourceDir+" something odd")